In [1]:
from anki.collection import Collection
from addon.application.services.formatter_service import (
    convert_note_to_addon_note,
)
from addon.domain.entities.note import AddonNote
from addon.domain.repositories.document_repository import (
    Document,
    SearchQuery,
)
from addon.infrastructure.persistence.qdrant_repository import (
    QdrantDocumentRepository,
)
from addon.application.use_cases.note_duplicate_finder import SimilarNoteFinder
from addon.domain.entities.note import AddonCollection


In [2]:
anki_collection_path = "/home/gianluca/.local/share/Anki2/User 1/collection.anki2"
anki_collection = Collection(anki_collection_path)

In [3]:
collection = AddonCollection("default")

note_ids = anki_collection.find_notes("")  # Empty query returns all notes
for note_id in note_ids:
    note = anki_collection.get_note(note_id)
    collection.add(convert_note_to_addon_note(note))

In [4]:
%%time
repository = QdrantDocumentRepository.create()

/home/gianluca/git/anki-addon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%%time
finder = SimilarNoteFinder(collection=collection, repository=repository)

CPU times: user 2min 53s, sys: 702 ms, total: 2min 54s
Wall time: 1min 30s


In [6]:
%%time
note = AddonNote(front="best way to avoid slow tests?", back="use fakes and nullables to prevent slow network calls or slow operations")
finder.find_duplicates(note)

CPU times: user 292 ms, sys: 16.9 ms, total: 309 ms
Wall time: 178 ms


[AddonNote(guid='Q~tIOur|Dj', front='When testing, how should we treat <i>slow</i> code? e.g., network calls or file system requests', back='We should execute it only when that behavior is explicitly part of the test of the unit under test. In every other case, we should use a Nullable', tags=['software-engineering', 'software-testing'], notetype=<AddonNoteType.BASIC: 'basic'>, deck_name=None)]